## Optional dataframe manipulation

In [2]:
import pandas as pd                 # dataframe storing
from bs4 import BeautifulSoup       # parsing web content in a nice way
import ssl                          # MAY BE UNNECESSARY: provides access to the security socket layer (ssl) https://docs.python.org/3/library/ssl.html
import urllib                       # MAY BE UNNECESSARY: open and navigate URL's
import os                           # Find where this file is located.
import time                         # for pausing the program in order for it to load the webpage
import re                           # regex for searching through strings


from selenium import webdriver      # load and run the webpage dynamically.


df = pd.read_excel('Data/text~Apple|PSF_web_data.xlsx')

# Turn the string of column OCM back into a list 
df['OCM'] = df.OCM.apply(lambda x: re.sub(" ",'',x))
df['OCM'] = df.OCM.apply(lambda x: x[1:-1].split(','))

# did it work? did it output a single OCM string?
df['OCM'][0][1]

'222'

In [3]:
# for searching by OCM in the list
lst = ["5310"]
msk = df['OCM'].apply(lambda x: not set(x).isdisjoint(lst))
out = df.loc[msk]
out

,Region,SubRegion,Culture,DocTitle,Year,OCM,OWC,Passage,run_Info
13,Africa,Northern Africa,Libyan Bedouin,Writing women's worlds: Bedouin stories,1993,"[5310, 533, 585, 902]",mt09,Apples from the gardens of the pious No one ea...,NaN
14,Africa,Northern Africa,Libyan Bedouin,Writing women's worlds: Bedouin stories,1993,"[5310, 593, 902]",mt09,Once there was a boy and girl whose mother die...,NaN
17,Africa,Northern Africa,Libyan Bedouin,Veiled sentiments: honor and poetry in a Bedou...,1986,"[157, 195, 5310]",mt09,The three conventional terms that are used to ...,NaN
473,South-America,Amazon and Orinoco,Saramaka,Two evenings in Saramaka,1991,"[171, 177, 212, 5310, 789, 852, 902]",sr15,Mató! 127 127. Aduêngi’s complex tale about a ...,NaN


### Running machine learning on OCM

In [4]:
# Make each OCM have its own row by exploding (you can reset the index with .reset_index(drop=True))
df = df.explode(column='OCM').reset_index(drop=True)
df

,Region,SubRegion,Culture,DocTitle,Year,OCM,OWC,Passage,run_Info
0,Africa,Central Africa,Azande,An account of the Zande,1926,137,fo07,The bagara looks like a miniature custard appl...,User: Eric
1,Africa,Central Africa,Azande,An account of the Zande,1926,222,fo07,The bagara looks like a miniature custard appl...,User: Eric
2,Africa,Central Africa,Azande,An account of the Zande,1926,252,fo07,The bagara looks like a miniature custard appl...,User: Eric
3,Africa,Central Africa,Azande,"Witchcraft, oracles and magic among the Azande",1937,787,fo07,It is operated in the following manner. A man ...,Run Time: 11:31:57
4,Africa,Central Africa,Azande,"Witchcraft, oracles and magic among the Azande",1937,754,fo07,‘She began by grinding sesame and when she had...,Run Date: 10/31/22
...,...,...,...,...,...,...,...,...,...
1503,South-America,Southern South America,Mataco,The Mataco Indians and their language,1897,192,si07,One notices here the constant change of u to a...,NaN
1504,South-America,Southern South America,Mataco,The Mataco Indians and their language,1897,196,si07,One notices here the constant change of u to a...,NaN
1505,South-America,Southern South America,Ona,Analytical and critical bibliography of the tr...,1917,104,sh04,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow...",NaN
1506,South-America,Southern South America,Ona,Analytical and critical bibliography of the tr...,1917,192,sh04,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow...",NaN


In [5]:
from mlxtend.preprocessing import TransactionEncoder

# We will use the apriori module to generate a dataframe that
# we can use for association rule finding
from mlxtend.frequent_patterns import apriori

# We will use the association_rules module to generate
# our association rules from the apriori output data frame
from mlxtend.frequent_patterns import association_rules





In [6]:
# df['Index_Col'] = df.index
df_smaller = df[['Culture', 'OCM','Passage']]
df_smaller

,Culture,OCM,Passage
0,Azande,137,The bagara looks like a miniature custard appl...
1,Azande,222,The bagara looks like a miniature custard appl...
2,Azande,252,The bagara looks like a miniature custard appl...
3,Azande,787,It is operated in the following manner. A man ...
4,Azande,754,‘She began by grinding sesame and when she had...
...,...,...,...
1503,Mataco,192,One notices here the constant change of u to a...
1504,Mataco,196,One notices here the constant change of u to a...
1505,Ona,104,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow..."
1506,Ona,192,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow..."


In [7]:
df_group = df_smaller.groupby(by = ['Culture', 'Passage'])
df_group

In [8]:
def make_OCM_list(x):

    '''
    Will return a list of the unique items
    in a particular grouping when used with
    the agg method as its function
    '''

    return x.unique()

In [9]:
# Use the agg method and make_OCM_list
# to return a list of unique items for eachmocm
df_OCMs = df_group.agg(make_OCM_list)

In [10]:
list_trans = list(df_OCMs['OCM'])
list_trans = list_trans[0:]
len(list_trans)

489

In [11]:
te = TransactionEncoder()
encoded_itemset = te.fit(list_trans).transform(list_trans)
print(encoded_itemset.shape) # show possible transcations and number of items
te.columns_



df_encoded = pd.DataFrame(encoded_itemset, columns = te.columns_)
df_encoded.head()

(489, 311)


,,0,101,102,103,104,105,113,114,115,...,872,876,877,881,883,885,887,900,901,902
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
# Before we begin, let's do a small
# amount of cleanup.  Let's remove all
# columns (items) that have less than 1 characters since that is just blank space
# more data cleaning my be required as time continues in case errors become evident in the scraped dataset
OCM_items = list(filter(lambda x: len(x) < 1, te.columns_ ))
print("removed: ",  OCM_items)
df_encoded = df_encoded.drop(columns=OCM_items) #remove small strings as they seem not to be items
print('How many unique items are left?', len(df_encoded.columns))

removed:  ['']
How many unique items are left? 310


In [13]:
# Use apriori to create a dataframe with columns of support and itemset lists
# Note that if your items are large compared to your sample (you have few rows but many columns) I reccommend using 
# a higher min_support as many more combinations may have spuriously higher support. Also, you can crash the program if too many are selected
df_support = apriori(df_encoded, min_support=0.005, use_colnames=True)
df_support.sort_values('support', inplace=True, ascending = False)
df_support

,support,itemsets
43,0.130879,(262)
38,0.102249,(245)
0,0.102249,(0)
20,0.085890,(177)
34,0.081800,(241)
...,...,...
232,0.006135,"(241, 242)"
233,0.006135,"(241, 244)"
235,0.006135,"(241, 251)"
242,0.006135,"(241, 464)"


## Use association_rules to find the rules

Using the dataframe generated by `apriori`, find the association rules with the greatest lift.  See the [association_rules documentation](https://rasbt.github.io/mlxtend/api_modules/mlxtend.frequent_patterns/association_rules/) for how to do this.

Sort the resulting DataFrame by lift in descending order.  A lift > 1 indicates that the items are often purchased together and that buying X will increase the purchase of Y.  A lift of < 1 indicates the items are often substituted.  That is X is substituted for Y so X and Y don't appear together often.

Examine the resulting DataFrame.  For the association rule X -> Y, X is the column `antecedents` and Y is the column `consequents`.  If sorted you can see the metrics for each rule based upon the lift.

In [14]:
# Find the association rules
rules = association_rules(df_support, metric = 'lift', min_threshold=1.0)
# lift >1 more likely than chance X means you see Y
# lift = 1 as often as chance
# lift <1 (substitution) less likely than chance X means you see Y


In [15]:
# Sort the rules by lift
# and examine the output
# to find what rules were
# discovered
rules.sort_values('lift', ascending=False, inplace =True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
513,"(462, 824)","(302, 415)",0.006135,0.006135,0.006135,1.000000,163.000000,0.006097,inf
867,"(462, 824, 222)","(302, 415)",0.006135,0.006135,0.006135,1.000000,163.000000,0.006097,inf
438,"(302, 462)","(415, 222)",0.006135,0.006135,0.006135,1.000000,163.000000,0.006097,inf
440,"(302, 222)","(462, 415)",0.006135,0.006135,0.006135,1.000000,163.000000,0.006097,inf
441,"(462, 415)","(302, 222)",0.006135,0.006135,0.006135,1.000000,163.000000,0.006097,inf
...,...,...,...,...,...,...,...,...,...
804,(796),(241),0.063395,0.081800,0.006135,0.096774,1.183065,0.000949,1.016579
42,(241),(262),0.081800,0.130879,0.012270,0.150000,1.146094,0.001564,1.022495
43,(262),(241),0.130879,0.081800,0.012270,0.093750,1.146094,0.001564,1.013187
21,(262),(245),0.130879,0.102249,0.014315,0.109375,1.069687,0.000933,1.008001


In [18]:
# look for OCM codes within the list
lst = frozenset(["241","226"])
msk = rules['antecedents'].apply(lambda x: not set(x).isdisjoint(lst))
out = rules.loc[msk]
out

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
426,"(241, 222)","(226, 462)",0.008180,0.006135,0.006135,0.750000,122.250000,0.006085,3.975460
427,"(226, 462)","(241, 222)",0.006135,0.008180,0.006135,1.000000,122.250000,0.006085,inf
412,"(241, 222)","(224, 462)",0.008180,0.008180,0.006135,0.750000,91.687500,0.006068,3.967280
411,"(241, 462)","(224, 222)",0.014315,0.006135,0.006135,0.428571,69.857143,0.006047,1.739264
424,"(241, 226)","(462, 222)",0.008180,0.012270,0.006135,0.750000,61.125000,0.006035,3.950920
...,...,...,...,...,...,...,...,...,...
39,(241),(245),0.081800,0.102249,0.012270,0.150000,1.467000,0.003906,1.056177
850,(241),(251),0.081800,0.053170,0.006135,0.075000,1.410577,0.001786,1.023600
148,(241),(824),0.081800,0.077710,0.008180,0.100000,1.286842,0.001823,1.024767
805,(241),(796),0.081800,0.063395,0.006135,0.075000,1.183065,0.000949,1.012546
